In [1]:
import pandas as pd
from top2vec import Top2Vec
from dataframe_generator import d2v_pandas_generator
import networkx as nx
from node2vec import Node2Vec
from gensim.models import Word2Vec
import numpy as np

In [2]:
d2v_model = Top2Vec.load("global_topic_model")
references_df = pd.read_pickle("pandas_data.pkl")
G=nx.read_graphml("citation_graph.gz")

In [ ]:
df=d2v_pandas_generator(d2v_model,references_df)
df=df[["d2v","id"]]

In [ ]:
#node2vec = Node2Vec(G, dimensions=100, walk_length=30, num_walks=3000, workers=4)
#n2v_model = node2vec.fit(window=10, min_count=1, batch_words=4)

#node2vec.wv.save_word2vec_format("embeddings.emb")
#node2vec.save("embeddings.model")

In [5]:
# Load model after Node2Vec.save

def n2v_df(row):
    try:
        return n2v_model.wv.get_vector(row['id'])
    except:
        return pd.NaT
n2v_model = Word2Vec.load("embeddings.model")
df["n2v"]=df.apply(lambda row: n2v_df(row), axis=1)
df=df.dropna()

In [ ]:
df["embedding"]=df.apply(lambda row: np.concatenate((row['d2v'], row['n2v']), axis=None), axis=1)

In [20]:
df=df.reset_index()
df=df[["id","d2v","n2v","embedding"]]

In [ ]:
df

In [22]:
df.to_pickle("embeddings_df.pkl")